In [34]:
from datasets import load_dataset

subset = "russe"
dataset = load_dataset("RussianNLP/russian_super_glue", subset)

dataset = dataset["validation"]
# dataset = dataset["test"]

Generating test split: 100%|██████████| 18892/18892 [00:01<00:00, 13051.21 examples/s]


In [35]:
dataset[1]

{'word': 'звание',
 'sentence1': 'Будьте во всем достойными звания советского партизана',
 'sentence2': 'Встретит вас гоф-фурьер такой-то (может быть, иначе называлось его звание – не помню)',
 'start1': 26,
 'start2': 67,
 'end1': 33,
 'end2': 74,
 'gold_sense1': 2,
 'gold_sense2': 1,
 'idx': 1,
 'label': 0}

In [36]:
dataset[10]

{'word': 'золото',
 'sentence1': 'Он взял здоровую кисть и золотой краской написал: «Смерть контрреволюции». Краска прошла сквозь материю, и буквы отпечатались золотом на обоях',
 'sentence2': 'Стоя по колено в бледно-голубой холодной воде, я целый день промывал золото',
 'start1': 126,
 'start2': 69,
 'end1': 134,
 'end2': 76,
 'gold_sense1': 2,
 'gold_sense2': 1,
 'idx': 10,
 'label': 0}

In [42]:
dataset[16]

{'word': 'евреи',
 'sentence1': 'Мать моя так хотела учиться, что пробилась на выучку к раввину, хотя евреи учат только мальчиков',
 'sentence2': 'Хотя считается, что у евреев должны быть черные волосы и карие глаза, среди них встречаются голубоглазые блондины',
 'start1': 69,
 'start2': 22,
 'end1': 75,
 'end2': 29,
 'gold_sense1': 1,
 'gold_sense2': 1,
 'idx': 16,
 'label': 1}

In [45]:
dataset[17]

{'word': 'добродетель',
 'sentence1': 'Отношения с хорошим и плохим, с пороками и добродетелью у художника чрезвычайно непростые',
 'sentence2': 'Добродетель гибнет',
 'start1': 43,
 'start2': 0,
 'end1': 56,
 'end2': 12,
 'gold_sense1': 1,
 'gold_sense2': 1,
 'idx': 17,
 'label': 1}

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from verbalist.generation.generation_utils import VerbalistConversation, generate

weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-25900/adapter_model"
tokenizer_path = "verbalist/model/models/verbalist_7b_v7/"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float16,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    trust_remote_code=True,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:36<00:00, 18.30s/it]


In [44]:
# inputs = ["Почему трава зеленая?"]

inputs = [
    f"""Предложение 1: Мать моя так хотела учиться, что пробилась на выучку к раввину, хотя евреи учат только мальчиков
Предложение 2: Хотя считается, что у евреев должны быть черные волосы и карие глаза, среди них встречаются голубоглазые блондины
Думай шаг за шагом. Является ли слово евреи одинаковым по значению и смыслу в этих двух предложениях. В ответе напиши 'да' или 'нет'.
"""
]

conversation = VerbalistConversation()
conversation.add_user_message(inputs[0])
prompt = conversation.get_prompt(tokenizer)
print("PROMPT", prompt)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=512,
    # no_repeat_ngram_size=15,
    repetition_penalty=1.1,
    temperature=0.5,
    top_k=40,
    top_p=0.95,
    # do_sample=True,
)
output = generate(model, tokenizer, prompt, generation_config)
# print(inp)
print(output)

PROMPT <s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Предложение 1: Мать моя так хотела учиться, что пробилась на выучку к раввину, хотя евреи учат только мальчиков
Предложение 2: Хотя считается, что у евреев должны быть черные волосы и карие глаза, среди них встречаются голубоглазые блондины
Думай шаг за шагом. Является ли слово евреи одинаковым по значению и смыслу в этих двух предложениях. В ответе напиши 'да' или 'нет'.
 </s> 
<s> bot


Нет, это не одно и то же слово. В первом предложении употребляется слово «еврей», а во втором - «евреи».


In [8]:
dataset[0]

{'sentence1': 'Кошка сидела на коврике.',
 'sentence2': 'Кошка не сидела на коврике.',
 'knowledge': '',
 'lexical-semantics': '',
 'logic': 'Negation',
 'predicate-argument-structure': '',
 'idx': 0,
 'label': 1}

In [52]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import json
from pathlib import Path


class RussianSuperGluePrompts:
    def lidirus_prompt(self, item):
        sentence1 = item["sentence1"]
        sentence2 = item["sentence2"]
        prompt = f"""Текст: "{sentence1}"\nИспользуя текст, можно ли сказать, что утверждение "{sentence2}" точно корректно относительно ситуации из текста? Ответь только "да" или "нет"."""
        return prompt

    def rucos_prompt(self, item):
        word_list = ", ".join([elem.strip() for elem in item["entities"]])
        text = item["passage"]
        prompt = f"""\nТекст: {text}\nЗапрос: {item['query']}\nСписок слов: {word_list}\nСогласно тексту, замени @placeholder запросе на наиболее подходящее слово из списка.\nВ качестве ответа верни только одно слово."""
        return prompt

    def muserc_prompt(self, item):
        prompt = f"""\nтекст: {item['paragraph']}\nвопрос: {item['question']}\nЯвляется ли "{item['answer']}" правильным ответом на этот вопрос? Думай шаг за шагом. Основываясь на тексте, ответь только "правильно" или "неправильно"."""
        return prompt

    def terra_prompt(self, item):
        prompt = f"""\nконтекст: {item['premise']}\nвывод: {item['hypothesis']}\nявляется ли вывод правильным исходя из контекста? Думай шаг за шагом. В ответе напиши только "правильный" или "неправильный"."""
        return prompt

    def danetqa_prompt(self, item):
        prompt = f"{item['question']}\nКонтекст: {item['passage']}\nИспользуя контекст, ответь на вопрос используя только да или нет."
        return prompt

    def parus_prompt(self, item):
        cause = "следствием " if item["cause"] == "effect" else "причиной"
        prompt = f"Текст: {item['premise']}\nвыбор 1: {item['choice1']}\nвыбор 2: {item['choice2']}\nДумай шаг за шагом. Выбери вариант который послужил {cause} для поля 'Текст'. В ответе напиши 'выбор 1' или 'выбор 2'."
        return prompt

    def russe_prompt(self, item):
        prompt = f"""Предложение 1: {item['sentence1']}\nПредложение 2: {item['sentence2']}\nДумай шаг за шагом. Является ли слово "{item['word']}" одинаковым по значению и смыслу в этих двух предложениях. В ответе напиши 'да' или 'нет'."""
        return prompt


class RussianSuperGlueEval:
    def lidirus_eval(self, item=None, result=None):
        answer = None
        if "да" in result:
            answer = 1
        elif "не" in result:
            answer = 0
        else:
            answer = int(not bool(item["label"]))

        return answer

    def rucos_eval(self, item=None, result=None):
        answer = int(item["answers"][0].lower() in result)

        return answer

    def muserc_eval(self, item=None, result=None):
        answer = None
        # result = result
        if "неправил" in result or "не правил" in result:
            answer = 0
        elif "правил" in result:
            answer = 1
        else:
            answer = int(not bool(item["label"]))

        return answer

    def terra_eval(self, item=None, result=None):
        answer = None
        # result = result
        if "неправил" in result:
            answer = 1
        elif "правил" in result:
            answer = 0
        else:
            answer = int(not bool(item["label"]))
        return answer

    def danetqa_eval(self, item=None, result=None):
        answer = None

        if "да" in result:
            answer = 1
        elif "не" in result:
            answer = 0
        else:
            answer = int(not bool(item["label"]))
        return answer

    def russe_eval(self, item=None, result=None):
        answer = None

        if "нет" in result:
            answer = 0
        elif "да" in result:
            answer = 1
        else:
            answer = int(not bool(item["label"]))
        return answer

    def parus_eval(self, item=None, result=None):
        answer = None

        if "1" in result:
            answer = 0
        elif "2" in result:
            answer = 1
        else:
            answer = int(not bool(item["label"]))
        return answer


class EvalRussianSuperGlue(RussianSuperGluePrompts, RussianSuperGlueEval):
    def __init__(
        self,
        dataset_name="danetqa",
        model_type=None,
        model=None,
        base_folder=None,
        eval_name=None,
        debug_mode=False,
    ) -> None:
        self.dataset_name = dataset_name
        self.dataset = load_dataset("RussianNLP/russian_super_glue", dataset_name)

        if dataset_name in ["lidirus"]:
            self.dataset = self.dataset["test"]
        else:
            self.dataset = self.dataset["validation"]

        self.model = model
        self.model_type = model_type

        self.base_folder = Path(base_folder)
        self.eval_name = eval_name

        self.debug_mode = debug_mode

        if self.debug_mode:
            num = 10
            self.dataset = self.dataset.select(range(20, 20 + num))

    def evaluate(self):
        task_name = self.dataset_name
        print(task_name)

        eval_folder = self.base_folder / f"{self.eval_name}"
        eval_folder.mkdir(exist_ok=True)
        output_file = eval_folder / f"{task_name}.jsonl"
        if output_file.is_file() and not self.debug_mode:
            print("score file")
            predicts = []
            ground_true = []
            with open(output_file, "r") as f:
                lines = f.readlines()
                for i, line in enumerate(lines):
                    line = json.loads(line)
                    line["label"] = int(line["label"])
                    predicts.append(line["label"])
                    gold_true = self.get_gold_true(item=dataset[i])
                    ground_true.append(dataset[i]["label"])
                acc = accuracy_score(ground_true, predicts)
                print(f"Accuracy: {acc}")
        else:
            predicts = []
            ground_true = []

            with open(eval_folder / f"{task_name}.log", "w") as f:
                idxs = []

                for item in tqdm(self.dataset):
                    prompt = self.get_prompt(item=item)
                    result = self.get_answer(prompt=prompt)

                    print(prompt, file=f)
                    print(f"predict answer = {result}", file=f)
                    print(f"real answer = {item}", file=f)

                    answer = self.evaluate_answer(item=item, result=result)
                    gold_true = self.get_gold_true(item=item)

                    predicts.append(answer)
                    ground_true.append(gold_true)
                    idxs.append(item["idx"])

                acc = str(accuracy_score(ground_true, predicts))
                print(f"Accuracy: {acc}")

                with open(output_file, "w") as f:
                    for idx, predict in zip(idxs, predicts):
                        answer = {
                            "idx": idx,
                            "label": predict,
                        }
                        json.dump(answer, f)
                        f.write("\n")
                with open(eval_folder / f"{task_name}.txt", "w") as f:
                    f.write(acc)

    def get_answer(self, prompt):
        models_map = {
            "verbalist": self.verbalist_generation_1,
        }
        answer = models_map[self.model_type](prompt)
        answer = answer.strip()
        answer = answer.lower()
        return answer

    def get_gold_true(self, item):
        handlers_map = {
            "lidirus": lambda item: item["label"],
            "rucos": lambda item: 1,
            "muserc": lambda item: item["label"],
            "terra": lambda item: item["label"],
            "danetqa": lambda item: item["label"],
            "parus": lambda item: item["label"],
            "russe": lambda item: item["label"],
        }
        return handlers_map[self.dataset_name](item=item)

    def get_prompt(self, item):
        handlers_map = {
            "lidirus": self.lidirus_prompt,
            "rucos": self.rucos_prompt,
            "muserc": self.muserc_prompt,
            "terra": self.terra_prompt,
            "danetqa": self.danetqa_prompt,
            "parus": self.parus_prompt,
            "russe": self.russe_prompt,
        }
        return handlers_map[self.dataset_name](item=item)

    def evaluate_answer(self, result, item):
        handlers_map = {
            "lidirus": self.lidirus_eval,
            "rucos": self.rucos_eval,
            "muserc": self.muserc_eval,
            "terra": self.terra_eval,
            "danetqa": self.danetqa_eval,
            "parus": self.parus_eval,
            "russe": self.russe_eval,
        }
        return handlers_map[self.dataset_name](item=item, result=result)

    def verbalist_generation_1(self, prompt):
        conversation = VerbalistConversation()
        conversation.add_user_message(prompt)
        prompt = conversation.get_prompt(tokenizer)
        # print("PROMPT", prompt)
        generation_config = GenerationConfig(
            bos_token_id=1,
            eos_token_id=2,
            pad_token_id=0,
            max_new_tokens=512,
            # no_repeat_ngram_size=15,
            repetition_penalty=1.1,
            temperature=0.5,
            top_k=40,
            top_p=0.95,
            # do_sample=True,
        )
        output = generate(
            self.model,
            tokenizer,
            prompt,
            generation_config,
        )
        # print("RESULT", output)
        return output


for name in ["lidirus", "rucos", "muserc", "terra", "danetqa", "parus", "russe"]:
    evaluation = EvalRussianSuperGlue(
        dataset_name=name,
        model_type="verbalist",
        model=model,
        base_folder="verbalist/evaluation/russian_super_glue/valid_evaluations/",
        eval_name="verbalist_7b_v7_checkpoint-25900",
        debug_mode=True,
    )

    evaluation.evaluate()

lidirus


  0%|          | 0/10 [00:00<?, ?it/s]/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kos